In [1]:
import os
import struct
import sys
from array import array
from os import path
from torch.autograd import Variable
import torch
import pandas as pd
import numpy as np
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from sklearn import preprocessing
import png
import numpy as np
!pip install torch 
import torch
from pytorchtools import Earlystopping
print(torch.__version__)

ModuleNotFoundError: No module named 'pytorchtools'

In [ ]:
X_train = pd.read_csv('c:/Data/X_train_first-100-cyc_60.csv', header= None).T
Y_train = pd.read_csv('c:/Data/Y_train_first-100-cyc_60.csv', header= None).T
Y_train_class = (Y_train >= 550)

save_path = 'c:/Data/output/FCC/Y_train_newdatasplit.csv'
np.savetxt(save_path, Y_train, delimiter=',')


#print ('Y_train_class.shape',Y_train_class.shape)
#print ('Y_train_class',Y_train_class)
#print ('X_train.shape', X_train.shape)

X_train = torch.tensor(X_train.values).float()
Y_train = torch.tensor(Y_train.values).float()
Y_train_class = Y_train_class *1
Y_train_class = torch.tensor(Y_train_class.values).float()


In [ ]:
X_dev = pd.read_csv('c:/Data/X_dev_first-100-cyc_20.csv', header= None).T
Y_dev = pd.read_csv('c:/Data/Y_dev_first-100-cyc_20.csv', header= None).T
X_dev = torch.tensor(X_dev.values).float()
Y_dev = torch.tensor(Y_dev.values).float()

save_path = 'c:/Data/output/FCC/Y_dev_newdatasplit3.csv'
np.savetxt(save_path, Y_dev, delimiter=',')


In [ ]:
X_test = pd.read_csv('c:/Data/X_test_first-100-cyc_20.csv', header= None).T
Y_test = pd.read_csv('c:/Data/Y_test_first-100-cyc_20.csv', header= None).T
X_test = torch.tensor(X_test.values).float()
Y_test = torch.tensor(Y_test.values).float()


save_path = 'c:/Data/output/FCC/Y_test_newdatasplit3.csv'
np.savetxt(save_path, Y_test, delimiter=',')


In [ ]:
X_train_norm = preprocessing.scale(X_train, axis=0, with_mean=True, with_std=True, copy=True)
X_train_norm = torch.tensor(X_train_norm).float()
#np.savetxt('./Data/X_train_norm.csv', X_train_norm, delimiter=',')

X_dev_norm = preprocessing.scale(X_dev, axis=0, with_mean=True, with_std=True, copy=True)
X_dev_norm = torch.tensor(X_dev_norm).float()
#np.savetxt('c:/Data/X_train_norm.csv', X_train_norm, delimiter=',')

X_test_norm = preprocessing.scale(X_test, axis=0, with_mean=True, with_std=True, copy=True)
X_test_norm = torch.tensor(X_test_norm).float()

In [ ]:
#Step 2: Define the model

#2-layer neural network (LINEAR->RELU->LINEAR->SOFTMAX)

import torch.nn as nn

import torch.nn.functional as F

model = nn.Sequential(
    #Flatten(),
    nn.Linear(800, 100),
    nn.ReLU(),
    nn.Linear(100, 50),
    nn.ReLU(),
    nn.Linear(50, 5),
    nn.ReLU(),
    nn.Linear(5, 5),
    nn.ReLU(),
    nn.Linear(5, 1),
    nn.ReLU())

In [ ]:
#Step 3: Define loss function and optimizer

#We will use the cross entropy loss and Adam optimizer

import torch.optim as optim

# Define the cost function
criterion = nn.MSELoss()

# Define the optimizer, learning rate 
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [ ]:
cost_per_iteration = []
for i in range(10000):
    # zero the parameter 
    optimizer.zero_grad()
    
    # forward propogatiom
    outputs = model(X_train)
    #print('outputs', outputs)
    #print('Y_train',Y_train)
    #print ('outputs', outputs)
    # calculate the loss
    loss = torch.sqrt(criterion(outputs, Y_train))
    print ('loss',loss)

    # backpropogation + update parameters
    loss.backward()
    optimizer.step()

    # print statistics
    cost = loss.item()
    cost_per_iteration.append(cost)
    if i % 100 == 0:  # print every 1000 iterations
        print('Epoch:' + str(i) + ", Iteration: " + str(i)
              + ", training cost = " + str(cost))
        #print('outputs: ', outputs)
        # for name, param in model.named_parameters():
        #     if param.requires_grad:
        #         print (name, param.data)
        
save_path = 'c:/Data/output/FCC/FCC_actualcyclelife_learning0.01.csv'
np.savetxt(save_path, cost_per_iteration, delimiter=',')


In [ ]:
def calculate_percentage_error(X, Y):
    outputs = model(X)
    Y_pred = outputs 
    
    Y_pred = Y_pred.float()
    divider = torch.abs(Y_pred - Y) 
    divider2 = divider / Y * 100
    outputsize = Y.shape[0]
    sum = torch.sum(divider2)
    error= sum / outputsize
    Y_pred = Y_pred.detach().numpy()
    print('Y_pred', Y_pred)

    #save_path = 'c:/Data/output/Y_pred1.csv'
    #np.savetxt(save_path, Y_pred, delimiter=',')
    
    return error

In [ ]:
train_percentage_error = calculate_percentage_error(X_train, Y_train)

outputs = model(X_train)
outputs = outputs.detach().numpy()
save_path = 'c:/Data/output/Y_pred_train_learningrate=0.01_newdataset2.csv'
np.savetxt(save_path, outputs, delimiter=',')
dev_percentage_error = calculate_percentage_error(X_dev, Y_dev)

outputs = model(X_dev)
outputs = outputs.detach().numpy()
save_path = 'c:/Data/output/Y_pred_dev_learningrate=0.01_newdataset2.csv'
np.savetxt(save_path, outputs, delimiter=',')

test_percentage_error = calculate_percentage_error(X_test, Y_test)

outputs = model(X_test)
outputs = outputs.detach().numpy()
save_path = 'c:/Data/output/Y_pred_dev_learningrate=0.01_newdataset2.csv'
np.savetxt(save_path, outputs, delimiter=',')


print('train_percentage_error:',train_percentage_error)
print('dev_percentage_error:',dev_percentage_error)
print('test_percentage_error:',test_percentage_error)
